v0.90

> TODO:
- bg_pos_raw_part1: load from folder, not from github
- run locally
- extract funcitons to .py files
- try to load with polars
- add descriptions and fix headers
- describe where this dataset came from
- run with IS_GUEST=True and LOAD_SAVED_DATA=True

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import warnings

import re
import time

from copy import copy, deepcopy

In [2]:
IS_GUEST = False
LOAD_SAVED_DATA = True

In [3]:
if not IS_GUEST:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    root_dir = "/content/drive/MyDrive/softuni/the-grammar-whisperer"
    bg_pos_raw_part1 = 'https://raw.githubusercontent.com/MirkaIvanova/datasets/refs/heads/main/bg/bg_pos_raw_part1.csv'
    bg_pos_raw_part2 = 'https://raw.githubusercontent.com/MirkaIvanova/datasets/refs/heads/main/bg/bg_pos_raw_part2.csv'

data_raw_dir = f"{root_dir}/data/raw"
data_clean_dir = f"{root_dir}/data/clean"
data_processed_dir = f"{root_dir}/data/processed"

bg_pos_raw_wiki1000_csv = f"{data_raw_dir}/wiki1000_plus_words.tsv"

Mounted at /content/drive


In [4]:
warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning) # 🧡

> File: Bulgarian Part Of Speech Dataset_alt,
       Bulgarian Part Of Speech Dataset_raw.csv <-- use this. I split into two to be smaller than 100MB and renamed to bg_pos_raw_part1.csv and bg_pos_raw_part2.csv.
Source: https://www.kaggle.com/datasets/auhide/bulgarian-part-of-speech-dataset


## Load raw data containing Bulgarian words and their part of speech

##### Load 2 CSVs and combine them



In [5]:
start_time = time.time()
df1 = pd.read_csv(bg_pos_raw_part1, sep='\t')
df2 = pd.read_csv(bg_pos_raw_part2, sep='\t')
print(f"Execution time: {time.time() - start_time} seconds")

Execution time: 6.033625364303589 seconds


> **Combine the two dataframes** vertically, the csv files were split
to less than 100MB in order to fit the github limit

In [6]:
df1.shape, df2.shape

((734614, 4), (613488, 4))

In [7]:
df = pd.concat([df1, df2], ignore_index=True)  # Combine and regenerate a new index
del df1 # so we don't accidentally refer to it in subsequent code
del df2

In [8]:
df.shape

(1348102, 4)

##### Remove duplicate rows

In [9]:
df.head(10)

,word,lemma,form,pos
0,а,а,основна форма,съюз
1,а,а,основна форма,съюз
2,аба,аба,основна форма,съществително име
3,аба,аба,ед.ч.,съществително име
4,абата,аба,"ед.ч., членувано",съществително име
5,аби,аба,мн.ч.,съществително име
6,абите,аба,"мн.ч., членувано",съществително име
7,Абаджиев,Абаджиев,основна форма,фамилно име или презиме
8,Абаджиев,Абаджиев,мъжко,фамилно име или презиме
9,Абаджиева,Абаджиев,женско,фамилно име или презиме


In [10]:
rows_before = df.shape[0]

df = df.drop_duplicates()

print(f"Removed {rows_before - df.shape[0]} rows from dataframe")

Removed 123607 rows from dataframe


> ##### Split column "form" into multiple columns

In [11]:
df.head()

,word,lemma,form,pos
0,а,а,основна форма,съюз
2,аба,аба,основна форма,съществително име
3,аба,аба,ед.ч.,съществително име
4,абата,аба,"ед.ч., членувано",съществително име
5,аби,аба,мн.ч.,съществително име


In [12]:
sorted(list(df.form.unique()))

['бройна форма',
 'винителен падеж',
 'винителен падеж, кратка форма',
 'дателен падеж',
 'дателен падеж, кратка форма',
 'деепричастие',
 'ед.ч.',
 'ед.ч., непълен член',
 'ед.ч., пълен член',
 'ед.ч., членувано',
 'ж.р., ед.ч.',
 'ж.р., ед.ч., членувано',
 'ж.р., мн.ч.',
 'ж.р., мн.ч., членувано',
 'женско',
 'звателна форма',
 'именителен падеж',
 'кратка форма',
 'м.р., ед.ч.',
 'м.р., ед.ч., непълен член',
 'м.р., ед.ч., пълен член',
 'м.р., мн.ч.',
 'м.р., мн.ч., членувано',
 'мин.деят.несв.прич., ед.ч., ж.р.',
 'мин.деят.несв.прич., ед.ч., ж.р., членувано',
 'мин.деят.несв.прич., ед.ч., м.р.',
 'мин.деят.несв.прич., ед.ч., м.р., непълен член',
 'мин.деят.несв.прич., ед.ч., м.р., пълен член',
 'мин.деят.несв.прич., ед.ч., ср.р.',
 'мин.деят.несв.прич., ед.ч., ср.р., членувано',
 'мин.деят.несв.прич., мн.ч.',
 'мин.деят.несв.прич., мн.ч., членувано',
 'мин.деят.св.прич., ед.ч., ж.р.',
 'мин.деят.св.прич., ед.ч., ж.р., членувано',
 'мин.деят.св.прич., ед.ч., м.р.',
 'мин.деят.св.пр

In [13]:
# make a backup
df_original = deepcopy(df)

In [14]:
# 🩷move to .py file
# map value like 'мин.деят.несв.прич., ед.ч., м.р., непълен член' to new columns 'gender', 'number', 'definite_article', 'participle', etc.
form_mapping = {
    "основна форма":          ("base", 1),
    "бройна форма":           ("count form", 1),
    "кратка форма":           ("short form", 1),
    "мъжколична форма":       ("masculine_personal_form", 1), # masculine personal form
    "приблизителен брой":     ("approximate_number", 1), # approximate number

    "ед.ч.":         ("number", 1),
    "мн.ч.":         ("number", 2),

    "мъжко":         ("gender", 1),
    "м.р.":          ("gender", 1),
    "женско":        ("gender", 2),
    "ж.р.":          ("gender", 2),
    "ср.р.":         ("gender", 3),

    "сег.вр.":       ("tense", 1), # present tense
    "мин.несв.вр.":  ("tense", 2), # past imperfective tense
    "мин.св.вр.":    ("tense", 3), # past perfective tense

    "1л.":           ("person", 1),
    "2л.":           ("person", 2),
    "3л.":           ("person", 3),

    "непълен член":  ("definite_article", 1),
    "пълен член":    ("definite_article", 2),
    "членувано":     ("definite_article", 3),

    "сег.деят.прич.":      ("participle", 1), # present active participle
    "мин.деят.св.прич.":   ("participle", 2), # past active perfective participle
    "мин.деят.несв.прич.": ("participle", 3), # past active imperfective participle
    "мин.страд.прич.":     ("participle", 4), # past passive participle
    "деепричастие":        ("participle", 5), # gerund/adverbial participle

    "именителен падеж":       ("case", 1), # nominative case
    "винителен падеж":        ("case", 2), # accusative case
    "дателен падеж":          ("case", 3), # dative case
    "звателна форма":         ("case", 4), # vocative case
    "повелително наклонение": ("case", 5), # imperative case
}

def expand_column_to_columns(df, original_column, value_to_column_mapping):
    # add new columns and initialize with zeroes
    new_columns = set(value[0] for value in value_to_column_mapping.values())
    for new_column in new_columns:
        df.loc[:, new_column] = np.zeros(len(df), dtype=int)

    # For each substring → (column, value), create the column and remove substring from the original column
    for substring, (col, val) in value_to_column_mapping.items():
        # Regex to match the exact token (accounting for commas/whitespace)
        pattern = rf'(?:^|\s*){re.escape(substring)}(?:\s*|$)'

        # If substring is found, set the value in the column to val
        df[col] = np.where(df[original_column].str.contains(pattern, regex=True, na=False), val, df[col])

        # Remove all matched occurrences from the original column
        df.loc[:, original_column] = df[original_column].str.replace(pattern, '', regex=True)

    # Clean up commas and whitespace in the original column
    df[original_column] = (df[original_column]
                .str.replace(r',+', ',', regex=True)   # collapse multiple commas
                .str.strip(' ,'))                      # strip leading/trailing commas/spaces

    return df

In [15]:
output_file = f'{data_processed_dir}/bg_vocabulary_v1.csv'
if not LOAD_SAVED_DATA: # runs for 100 sec
    df = expand_column_to_columns(df, 'form', form_mapping)
    df = df.drop(columns=['form'])
    df.to_csv(output_file, index=False)
else:
    df = pd.read_csv(output_file)

In [16]:
assert df.shape == (1224495, 15) # 💙print with color on success

In [17]:
!ls -lh {data_processed_dir}

total 9.8G
-rw------- 1 root root  76M Jan 21 19:07 bg_vocabulary_final.csv
-rw------- 1 root root  96M Jan 21 14:05 bg_vocabulary_v1.csv
-rw------- 1 root root 663M Jan 21 15:54 sent_wikipedia_nlp_features_final.csv
-rw------- 1 root root 100M Jan 20 09:50 sent_wikipedia_nlp_features.part01.rar
-rw------- 1 root root  56M Jan 20 09:50 sent_wikipedia_nlp_features.part02.rar
-rw------- 1 root root 8.4M Jan 23 19:23 sent_wikipedia_nlp_features_stanza_final_10000_tmp.csv
-rw------- 1 root root 649M Jan 23 19:22 sent_wikipedia_nlp_features_stanza_final.csv
-rw------- 1 root root 1.1G Jan 23 08:34 sent_wikipedia_nlp_features_stanza_v1_checkpoint80.csv
-rw------- 1 root root 1.1G Jan 23 08:35 sent_wikipedia_nlp_features_stanza_v1.csv
-rw------- 1 root root 1.2G Jan 23 15:50 sent_wikipedia_nlp_features_stanza_v2.csv
-rw------- 1 root root 1.3G Jan 21 11:11 sent_wikipedia_nlp_features_v1.csv
-rw------- 1 root root 1.4G Jan 21 15:48 sent_wikipedia_nlp_features_v2.csv
-rw------- 1 root root 1.5G

###### Encode column 'pos'

In [18]:
sorted(list(df.pos.unique()))

['българско географско понятие',
 'българско населено място',
 'въпросително местоимение',
 'глагол',
 'държава',
 'лично име',
 'лично местоимение',
 'междуметие',
 'месец',
 'наречие',
 'неопределително местоимение',
 'обобщително местоимение',
 'относително местоимение',
 'отрицателно местоимение',
 'показателно местоимение',
 'предлог',
 'прилагателно име',
 'притежателно местоимение',
 'световноизвестен град',
 'световноизвестно географско понятие',
 'столица',
 'съществително име',
 'съюз',
 'търговска марка',
 'фамилно име или презиме',
 'частица',
 'числително бройно име',
 'числително редно име']

In [19]:
# Define a custom mapping
pos_mapping = {
    'глагол':1,
    'съществително име':2,
    'прилагателно име':3,
    'наречие':4,
    'българско географско понятие':5,
    'българско населено място':6,
    'въпросително местоимение':7,
    'държава':8,
    'лично име':9,
    'лично местоимение':10,
    'междуметие':11,
    'месец':12,
    'неопределително местоимение':13,
    'обобщително местоимение':14,
    'относително местоимение':15,
    'отрицателно местоимение':16,
    'показателно местоимение':17,
    'предлог':18,
    'притежателно местоимение':19,
    'световноизвестен град':20,
    'световноизвестно географско понятие':21,
    'столица':22,
    'съюз':23,
    'търговска марка':24,
    'фамилно име или презиме':25,
    'частица':26,
    'числително бройно име':27,
    'числително редно име':28
 }

# Map the values
df['pos_encoded'] = df['pos'].map(pos_mapping)
print(df[['pos', 'pos_encoded']])

                             pos  pos_encoded
0                           съюз           23
1              съществително име            2
2              съществително име            2
3              съществително име            2
4              съществително име            2
...                          ...          ...
1224490        съществително име            2
1224491  фамилно име или презиме           25
1224492  фамилно име или презиме           25
1224493  фамилно име или презиме           25
1224494          търговска марка           24

[1224495 rows x 2 columns]


###### Convert to lowercase

In [33]:
columns_to_lower = ['word', 'lemma']
df[columns_to_lower] = df[columns_to_lower].apply(lambda x: x.str.lower())

###### Drop columns with repeated data

In [28]:
df = df.drop(columns=['pos'])

In [21]:
if not LOAD_SAVED_DATA:
    output_file = f'{data_processed_dir}/bg_vocabulary_final.csv'
    df.to_csv(output_file, index=False)
else:
    df = pd.read_csv(output_file)

###### Experiments...

In [40]:
df[(df.lemma=='социалдемократически') & (df.definite_article==1)]

,word,lemma,definite_article,number,gender,count form,approximate_number,base,short form,participle,masculine_personal_form,tense,case,person,pos_encoded


In [38]:
df.case.unique()

array([0, 4, 5, 1, 2, 3])